This code calculates manageable, vulnerable, and irreplaceable carbon across Y2Y

In [1]:
# import packages
import ee
import geemap

import pandas as pd

In [2]:
# authenticate the EE api
# ee.Authenticate()

In [3]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

## Define GEE datasets

In [61]:
# define EE datasets
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")
# countries = ee.FeatureCollection("USDOS/LSIB/2017")
# y2y_ecoregions = ee.FeatureCollection(
#     "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
# y2y_protected_areas = ee.FeatureCollection(
#     "projects/ee-bermane/assets/y2y_protected_areas")
# y2y_protected_areas_canada = ee.FeatureCollection(
#     "projects/ee-bermane/assets/y2y_protected_areas_canada")
rra = ee.FeatureCollection("projects/ee-bermane/assets/ross_river_ipca")
soc_olm = ee.Image("projects/ee-bermane/assets/soc_0_1m_kg_m2_olm")
# soc_soilgrids = ee.Image("projects/soilgrids-isric/ocs_mean")
soc_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")
# soc_socs = ee.Image("projects/ee-bermane/assets/SOCS_0_1m_mg_ha_year_2010")
peat = ee.Image("projects/sat-io/open-datasets/GLOBAL-PEATLAND-DATABASE")

## Calculate manageable biomass stock from ESA CCI data

Includes all biomass except for non-forest pixels within the desert biome and the following LC types: barren ground, snow/ice, water, cropland, urban and built-up.

## NEED TO MASK PROTECTED AREAS!

In [50]:
# calc 2021 ESA CCI biomass
# grab the 2021 AGB images
agb_2021 = agb.filter(ee.Filter.stringContains("system:index", "2021")).first()

# create image for litter using Harris ratio (4%)
# leave out dead wood since it is likely sensed already
# mask litter only for forested LC types
# 1: "#033e00",  # Temperate or sub-polar needleleaf forest
# 2: "#939b71",  # Sub-polar taiga needleleaf forest
# 3: "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
# 4: "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
# 5: "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
# 6: "#6b7d2c",  # Mixed forest

litter_mask = lc.gte(1).And(lc.lte(6))

litter = agb_2021.select(['AGB']).multiply(0.04).multiply(litter_mask)

# create image for BGB (using global rsr map)
bgb = agb_2021.select(['AGB']).multiply(rsr).rename('BGB')

# add AGB + litter and BGB together
bio_2021 = agb_2021.select(['AGB']).add(litter).addBands(
    bgb).rename(['agb_t_ha', 'bgb_t_ha'])

# multiply values by 0.47 to get carbon density
# 0.47 used by Harris et al. (2021)
# mask barren ground, snow/ice, water, cropland, urban and built-up
bio_2021 = bio_2021.multiply(0.47).updateMask(
    lc.neq(18).And(lc.neq(19)).And(lc.neq(16)).And(lc.neq(15)).And(lc.neq(17)))

# Rasterize y2y_biomes to extract desert biome
biomes_raster = y2y_biomes.reduceToImage(properties=['BIOME_ID'], reducer=ee.Reducer.first())

# create a mask of forest lc types
forest_mask = lc.gte(1).And(lc.lte(6))

# combine desert mask and forest mask
# desert BIOME_ID = 13 in biomes_raster
desert_mask = biomes_raster.eq(13).And(forest_mask.Not())

# mask desert pixels from bio_2021
bio_2021 = bio_2021.updateMask(desert_mask.Not())

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Create carbon layer mask to filter pixel area raster
carbon_mask = bio_2021.select(['agb_t_ha']).mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_agb_extent = pixel_area_ha.updateMask(carbon_mask)

# # calculate total biomass c per pixel and rename bands
# bio_stock_2021 = bio_2021.multiply(
#     pixel_area_ha
# ).rename(
#     ['agb_t', 'bgb_t']
# ).addBands(
#     pixel_area_agb_extent.rename(['pixel_area_agb_extent_ha'])
# ).addBands(
#     pixel_area_ha.rename(['pixel_area_ha'])
# )

# rename total manageable biomass carbon
manageable_bio_stock = bio_2021.multiply(
    pixel_area_ha
).rename(
    ['agb_t', 'bgb_t']
)

## Calculate manageable soil carbon stock from Sothe Canada data and Open Land Map US data

Includes all SOC 0-1m except for non-forest pixels within the desert biome, barren lands within temperate zones, and the following LC types: snow/ice, water, cropland, urban and built-up.

## NEED TO MASK PROTECTED AREAS!!!

In [83]:
# calc open land map soc
# multiply by 10 to get t/ha
# mask water/snow/ice/cropland, urban and built-up
soc_olm = soc_olm.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)).And(lc.neq(15)).And(lc.neq(17)))

# multiply by pixel area to get total carbon per pixel
soc_olm_stock = soc_olm.multiply(pixel_area_ha).rename('total_olm_soc')

# reproject to match sothe to blend images
soc_olm_reproj_sothe = soc_olm.resample('bilinear').toFloat()

In [84]:
# calc sothe soc
# multiply by 10 to get t/ha
# mask water/snow/ice/cropland, urban and built-up
soc_sothe = soc_sothe.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)).And(lc.neq(15)).And(lc.neq(17)))

# blend sothe and olm carbon across y2y
soc_blend = ee.ImageCollection([soc_olm_reproj_sothe, soc_sothe]).mosaic().rename('soc_t_ha').reproject(
    crs=soc_sothe.projection(),
    crsTransform=soc_sothe.projection().getInfo().get('transform')
).updateMask(lc.neq(18).And(lc.neq(19)).And(lc.neq(15)).And(lc.neq(17)))  # mask snow/ice, water, cropland, urban and built-up

# mask desert pixels from soc_blend
soc_blend = soc_blend.updateMask(desert_mask.Not())

# create mask for temperate barren lands 
# LC type (16)
# and biomes mask (5, 8)
temperate_barren_mask = lc.eq(16).And(biomes_raster.eq(5).Or(biomes_raster.eq(8)))
soc_blend = soc_blend.updateMask(temperate_barren_mask.Not())

# Create soc layer mask to filter pixel area raster
soc_mask = soc_blend.mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_soc_extent = pixel_area_ha.updateMask(soc_mask)

# # multiply by pixel area to get total carbon per pixel
# soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t').addBands(
#     pixel_area_soc_extent.rename(['pixel_area_soc_extent_ha'])
# ).addBands(
#     pixel_area_ha.rename(['pixel_area_ha'])
# )

# rename total manageable soil carbon
manageable_soc_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t')

## Calculate vulnerable biomass carbon stock
All manageable biomass carbon is vulnerable, so no calculations needed.

In [56]:
# rename total vulnerable biomass carbon
vulnerable_bio_stock = manageable_bio_stock

## Calculate vulnerable SOC stock
All SOC stock is vulnerable down to 100cm, but with varying percentages based on depth, lc type, and most likely conversion

Landcover key:
* 1: "Temperate or sub-polar needleleaf forest",
* 2: "Sub-polar taiga needleleaf forest",
* 3: "Tropical or sub-tropical broadleaf evergreen forest",
* 4: "Tropical or sub-tropical broadleaf deciduous forest",
* 5: "Temperate or sub-polar broadleaf deciduous forest",
* 6: "Mixed forest",
* 7: "Tropical or sub-tropical shrubland",
* 8: "Temperate or sub-polar shrubland",
* 9: "Tropical or sub-tropical grassland",
* 10: "Temperate or sub-polar grassland",
* 11: "Sub-polar or polar shrubland-lichen-moss",
* 12: "Sub-polar or polar grassland-lichen-moss",
* 13: "Sub-polar or polar barren-lichen-moss",
* 14: "Wetland",
* 15: "Cropland",
* 16: "Barren land",
* 17: "Urban and built-up",
* 18: "Water",
* 19: "Snow and ice"

In [ ]:
# rename total vulnerable soil carbon
vulnerable_soc_stock = manageable_soc_stock

# Rasterize y2y_biomes to use for ecosystem classification based on BIOME_ID:
# 5: Temperate Conifer Forests
# 6: Boreal Forests/Taiga
# 8: Temperate Grasslands, Savannas, and Shrublands
# 11: Tundra
# 13: Deserts and Xeric Shrublands
biomes_raster = y2y_biomes.reduceToImage(properties=['BIOME_ID'], reducer=ee.Reducer.first())

# ALL FORESTS --> FORESTRY
# create a mask based only on forest lc types
forest_mask = lc.gte(1).And(lc.lte(6))

# TEMPERATE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_grass_shrub_mask = lc.eq(8).Or(lc.eq(10)).And(peat.neq(1).And(peat.neq(2))).And(biomes_raster.eq(5).Or(biomes_raster.eq(8)))

# NORTHERN GRASSLANDS AND SHRUBLANDS --> MINING/OIL/GAS/DEV
# create a mask based on grasslands, shrublands, lichen-moss LC type (8, 10, 11, 12, 13)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_grass_shrub_mask = lc.eq(8).Or(lc.eq(10)).Or(lc.eq(11)).Or(lc.eq(12)).Or(lc.eq(13)).And(peat.neq(1).And(peat.neq(2))).And(biomes_raster.eq(6).Or(biomes_raster.eq(11)))

# MONTANE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
#???????

# TEMPERATE WETLANDS --> AGRICULTURE
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_wetlands_mask = lc.eq(14).And(peat.neq(1).And(peat.neq(2))).And(biomes_raster.eq(5).Or(biomes_raster.eq(8)))

# NORTHERN WETLANDS --> MINING/OIL/GAS/DEV
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_wetlands_mask = lc.eq(14).And(peat.neq(1).And(peat.neq(2))).And(biomes_raster.eq(6).Or(biomes_raster.eq(11)))

# TEMPERATE PEATLANDS --> AGRICULTURE
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (5, 8)
temperate_peatlands_mask = peat.eq(1).Or(peat.eq(2)).And(biomes_raster.eq(5).Or(biomes_raster.eq(8)))

# NORTHERN PEATLANDS --> MINING/OIL/GAS/DEV
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (6, 11)
northern_peatlands_mask = peat.eq(1).Or(peat.eq(2)).And(biomes_raster.eq(6).Or(biomes_raster.eq(11)))

# NORTHERN BARREN LANDS --> MINING/OIL/GAS/DEV
# create a mask based on barren lands LC type (16)
# and biomes mask (6, 11)
northern_barren_mask = lc.eq(16).And(biomes_raster.eq(6).Or(biomes_raster.eq(11)))

# set vulnerable SOC based on different masks
# forest_mask --> 0%
#vulnerable_soc_stock = vulnerable_soc_stock.where(forest_mask, 0)

check = lc.eq(13)

## Add layers to map

In [ ]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = [
    "#033e00",  # Temperate or sub-polar needleleaf forest
    "#939b71",  # Sub-polar taiga needleleaf forest
    "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    "#6b7d2c",  # Mixed forest
    "#b29d29",  # Tropical or sub-tropical shrubland
    "#b48833",  # Temperate or sub-polar shrubland
    "#e9da5d",  # Tropical or sub-tropical grassland
    "#e0cd88",  # Temperate or sub-polar grassland
    "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    "#bad292",  # Sub-polar or polar grassland-lichen-moss
    "#3f8970",  # Sub-polar or polar barren-lichen-moss
    "#6ca289",  # Wetland
    "#e6ad6a",  # Cropland
    "#a9abae",  # Barren land
    "#db2126",  # Urban and built-up
    "#4c73a1",  # Water
    "#fff7fe",  # Snow and ice
]

In [85]:
# create a map
m = geemap.Map()

# add layers
# m.addLayer(desert_mask, {}, "Desert Mask")
m.addLayer(y2y_biomes, {}, "Biomes")
m.addLayer(lc.clip(y2y), {"min": 1, "max": 19,
                          "palette": lc_palette}, 'Landcover')
# m.addLayer(pixel_area_agb_extent.clip(y2y), {}, 'Carbon Mask')
# m.addLayer(manageable_bio_stock.select(['agb_t']).clip(y2y), {
#     "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB')
m.addLayer(soc_blend, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Density Blend 0-1m")
# m.addLayer(pixel_area_soc_extent.clip(y2y), {}, 'Carbon Mask')
# m.addLayer(peat.clip(y2y), {}, "Peat Distribution")
# m.addLayer(rra, {}, "RRA")
# m.addLayer(vulnerable_soc_stock, {"min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable SOC Stock")
# m.addLayer(temperate_grass_shrub_mask, {}, "Temperate Grasslands and Shrublands")

# display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…